- #### Challenge 1: Generate invoice reports
Adventure Works Cycles sells directly to retailers, who must be invoiced for their orders. You have been tasked with writing a query to generate a list of invoices to be sent to customers.

1. Retrieve customer orders
As an initial step towards generating the invoice report, write a query that returns the company name from the SalesLT.Customer table, and the sales order ID and total due from the SalesLT.SalesOrderHeader table.
2. Retrieve customer orders with addresses
Extend your customer orders query to include the Main Office address for each customer, including the full street address, city, state or province, postal code, and country or region
Tip: Note that each customer can have multiple addressees in the SalesLT.Address table, so the database developer has created the SalesLT.CustomerAddress table to enable a many-to-many relationship between customers and addresses. Your query will need to include both of these tables, and should filter the results so that only Main Office addresses are included.

In [ ]:
--- 1.
SELECT c.CompanyName, oh.SalesOrderID, oh.TotalDue
FROM SalesLT.Customer AS c
JOIN SalesLT.SalesOrderHeader AS oh
    ON oh.CustomerID = c.CustomerID;

In [ ]:
---- 2.
SELECT c.CompanyName,
       a.AddressLine1,
       ISNULL(a.AddressLine2, '') AS AddressLine2,
       a.City,
       a.StateProvince,
       a.PostalCode,
       a.CountryRegion,
       oh.SalesOrderID,
       oh.TotalDue
FROM SalesLT.Customer AS c
JOIN SalesLT.SalesOrderHeader AS oh
    ON oh.CustomerID = c.CustomerID
JOIN SalesLT.CustomerAddress AS ca
    ON c.CustomerID = ca.CustomerID
JOIN SalesLT.Address AS a
    ON ca.AddressID = a.AddressID
WHERE ca.AddressType = 'Main Office'

### Challenge 2: Retrieve customer data
As you continue to work with the Adventure Works customer and sales data, you must create queries for reports that have been requested by the sales team.

1. Retrieve a list of all customers and their orders
The sales manager wants a list of all customer companies and their contacts (first name and last name), showing the sales order ID and total due for each order they have placed. Customers who have not placed any orders should be included at the bottom of the list with NULL values for the order ID and total due.
2. Retrieve a list of customers with no address
A sales employee has noticed that Adventure Works does not have address information for all customers. You must write a query that returns a list of customer IDs, company names, contact names (first name and last name), and phone numbers for customers with no address stored in the database.

In [ ]:
--- 1.
SELECT c.CompanyName, c.FirstName, c.LastName,
       oh.SalesOrderID, oh.TotalDue
FROM SalesLT.Customer AS c
LEFT JOIN SalesLT.SalesOrderHeader AS oh
    ON c.CustomerID = oh.CustomerID
ORDER BY oh.SalesOrderID DESC;

In [ ]:
--- 2.
SELECT c.CompanyName, c.FirstName, c.LastName,
       oh.SalesOrderID, oh.TotalDue
FROM SalesLT.Customer AS c
LEFT JOIN SalesLT.SalesOrderHeader AS oh
    ON c.CustomerID = oh.CustomerID
ORDER BY oh.SalesOrderID DESC;


### Challenge 3: Create a product catalog
The marketing team has asked you to retrieve data for a new product catalog.

1. Retrieve product information by category
- The product catalog will list products by parent category and subcategory, so you must write a query that retrieves the parent category name, subcategory name, and product name fields for the catalog.

In [ ]:
SELECT pcat.Name AS ParentCategory, cat.Name AS SubCategory, prd.Name AS ProductName
FROM SalesLT.ProductCategory pcat
JOIN SalesLT.ProductCategory as cat
    ON pcat.ProductCategoryID = cat.ParentProductCategoryID
JOIN SalesLT.Product as prd
    ON prd.ProductCategoryID = cat.ProductCategoryID
ORDER BY ParentCategory, SubCategory, ProductName;

## Subconsultas 

Challenge 1: Retrieve product price information
Adventure Works products each have a standard cost price that indicates the cost of manufacturing the product, and a list price that indicates the recommended selling price for the product. This data is stored in the SalesLT.Product table. Whenever a product is ordered, the actual unit price at which it was sold is also recorded in the SalesLT.SalesOrderDetail table. You must use subqueries to compare the cost and list prices for each product with the unit prices charged in each sale.

1. Retrieve products whose list price is higher than the average unit price.
2. Retrieve the product ID, name, and list price for each product where the list price is higher than the average unit price for all products that have been sold.
Tip: Use the AVG function to retrieve an average value.
Retrieve Products with a list price of 100 or more that have been sold for less than 100.
Retrieve the product ID, name, and list price for each product where the list price is 100 or more, and the product has been sold for less than 100.

In [ ]:
--- 1.
SELECT ProductID,Name,ListPrice
FROM SalesLT.Product
WHERE ListPrice > 
    (SELECT AVG(UnitPrice)    
     FROM SalesLT.SalesOrderDetail)
ORDER BY ProductID

In [ ]:
--- 2. 
SELECT ProductID, Name, ListPrice
FROM SalesLT.Product
WHERE ProductID IN
    (SELECT ProductID
     FROM SalesLT.SalesOrderDetail
     WHERE UnitPrice < 100.00)
AND ListPrice >= 100.00
ORDER BY ProductID;

### Challenge 2: Analyze profitability
The standard cost of a product and the unit price at which it is sold determine its profitability. You must use correlated subqueries to compare the cost and average selling price for each product.

1. Retrieve the cost, list price, and average selling price for each product
 Retrieve the product ID, name, cost, and list price for each product along with the average unit price for which that product has been sold.
2.  Retrieve products that have an average selling price that is lower than the cost.
Filter your previous query to include only products where the cost price is higher than the average selling price.

In [ ]:
---1.
SELECT p.ProductID, p.Name, p.StandardCost, p.ListPrice,
    (SELECT AVG(o.UnitPrice)
     FROM SalesLT.SalesOrderDetail AS o
     WHERE p.ProductID = o.ProductID) AS AvgSellingPrice
FROM SalesLT.Product AS p
ORDER BY p.ProductID;

In [ ]:
---2.
SELECT p.ProductID, p.Name, p.StandardCost, p.ListPrice,
    (SELECT AVG(o.UnitPrice)
    FROM SalesLT.SalesOrderDetail AS o
    WHERE p.ProductID = o.ProductID) AS AvgSellingPrice
FROM SalesLT.Product AS p
WHERE StandardCost >
    (SELECT AVG(od.UnitPrice)
     FROM SalesLT.SalesOrderDetail AS od
     WHERE p.ProductID = od.ProductID)
ORDER BY p.ProductID;

##  Uso de funciones integradas y GROUP BY en Transact-SQL

Challenge 1: Retrieve order shipping information
The operations manager wants reports about order shipping based on data in the SalesLT.SalesOrderHeader table.

1. Retrieve the order ID and freight cost of each order.
   - Write a query to return the order ID for each order, together with the the Freight value rounded to two decimal places in a column named FreightCost.
2. Add the shipping method.
   - Extend your query to include a column named ShippingMethod that contains the ShipMethod field, formatted in lower case.
3. Add shipping date details.
   - Extend your query to include columns named ShipYear, ShipMonth, and ShipDay that contain the year, month, and day of the ShipDate. The ShipMonth value should be displayed as the month name (for example, June) 

In [ ]:
--1.
SELECT SalesOrderID,ROUND(Freight,2) 
FROM Sales.SalesOrderHeader
--2. 
SELECT SalesOrderID,ROUND(Freight,2) [FreightCost],LOWER(B.Name)
FROM Sales.SalesOrderHeader AS A
INNER JOIN Purchasing.ShipMethod AS B 
ON A.ShipMethodID=B.ShipMethodID

---3.
SELECT SalesOrderID,ROUND(Freight,2) [FreightCost],LOWER(B.Name),
YEAR(ShipDate) ShipYear,DATENAME(mm,ShipDate) ShipMonth,DAY(ShipDate) ShipDay
FROM Sales.SalesOrderHeader AS A
INNER JOIN Purchasing.ShipMethod AS B 
ON A.ShipMethodID=B.ShipMethodID


Challenge 2: Aggregate product sales
The sales manager would like reports that include aggregated information about product sales.

1. Retrieve total sales by product
   - Write a query to retrieve a list of the product names from the SalesLT.Product table and the total revenue for each product calculated as the sum of LineTotal from the SalesLT.SalesOrderDetail table, with the results sorted in descending order of total revenue.
2. Filter the product sales list to include only products that cost over 1,000
   - Modify the previous query to include sales totals for products that have a list price of more than 1000.
3. Filter the product sales groups to include only total sales over 20,000
   - Modify the previous query to only include only product groups with a total sales value greater than 20,000.

In [ ]:
--1.
SELECT A.Name as Product, SUM(LineTotal) AS TotalRevenue
FROM Production.Product AS A 
INNER JOIN Sales.SalesOrderDetail AS B 
    ON A.ProductID=B.ProductID
GROUP BY A.Name
ORDER BY TotalRevenue DESC


--2.
SELECT A.Name as Product, SUM(LineTotal) AS TotalRevenue
FROM Production.Product AS A 
INNER JOIN Sales.SalesOrderDetail AS B 
    ON A.ProductID=B.ProductID
GROUP BY A.Name
ORDER BY TotalRevenue DESC

--3.

SELECT A.Name as Product, SUM(LineTotal) AS TotalRevenue
FROM Production.Product AS A 
INNER JOIN Sales.SalesOrderDetail AS B 
    ON A.ProductID=B.ProductID
WHERE A.ListPrice > 1000
GROUP BY A.Name
HAVING SUM(LineTotal) > 20000
ORDER BY TotalRevenue DESC